In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
base_path = '../input/siim-covid19-detectionut'
for dirname, _, filenames in os.walk(base_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!conda install -c conda-forge gdcm -y # prevnding GDCM errors when exporting images

# Resizeing and Converting Imgaes

In [ ]:
import pydicom
from pydicom import dcmread
from pydicom.data import get_testdata_file
import scipy.misc
from PIL import Image
import matplotlib.pyplot as plt
import cv2

# need to re-run once reload this note book
!pip install pylibjpeg pylibjpeg-libjpeg pydicom # handling Pixel Data that hasn't been compressed as some dcm files give runtime errors
import pylibjpeg

# Gathering Path

In [ ]:
trainimlist = []

for dirname, _, filenames in os.walk('../input/siim-covid19-detection/train'):
    for filename in filenames:
        trainimlist.append(os.path.join(dirname, filename))

# Create Save Path

In [ ]:
!mkdir "/kaggle/working/output" 

In [ ]:
image_size = pd.DataFrame(columns = ['id', 'ori_width', 'ori_height', 're_width', 're_height']) # to save resized image data

for pic in trainimlist:
    data = []
    # preparing save files 
    save_pic = pic.replace("../input/siim-covid19-detection/train", '')
    root = save_pic.split('/')
    save_root = '/kaggle/working/output/'+root[1]+'/'+root[2]+'/'
    
    if os.path.exists(save_root) == False: # some images have the same study_id (path)
        os.makedirs(save_root)    
    
    # reading images
    Input_Image = pydicom.dcmread(pic) # class type
    
    # can check meta-DCM data
    # print(Input_Image)
    
    img_height  = Input_Image.Rows
    img_width  = Input_Image.Columns
    
    Input_Image.PhotometricInterpretation = 'YBR_FULL' # handle GDCM and Pillow(for jpg, jpeg)
    img = Input_Image.pixel_array  # type is 2d np.array
    
    new_path = save_root + root[3].replace('.dcm','.jpg') # creating save file as jpg
    cv2.imwrite(new_path,img) # convert to jpg (ndarray)    
    
    img = (np.maximum(img, 0) / img.max())*255 # Normalize the image between 0 and 255 (colors)
    img.astype(np.uint8) # float values need to be int for the next step
    img_mem = Image.fromarray(img) # back to the image (cannot handle float values)
    img_mem = img_mem.convert("L") # L -> 8 bit : RGB -> 24 bits
   
    data.append(root[3].replace('.dcm','.jpg'))
    data.append(float(img_width))
    data.append(float(img_height))
    
    # resize image sizes as 416: keeping aspect ratio    
    fixed_size = 416 # modify this size 
    
    # if keep the aspect (YOLO needs width and heoght can be *32)
    # height_percent = (fixed_size / float(img_height)) # img_mem.size[1] is height
    # width_size = int((float(img_width) * float(height_percent)))

    img_mem = img_mem.resize((416, 416), Image.NEAREST)
   
    # plt.imshow(img_mem)
    
    data.append(416)
    data.append(416)
    
    # saving the images as jpg
    img_mem.save(new_path)
    
    index = len(image_size) # geting index number to add                
    image_size.loc[index] = data # adding the row    
        

image_size.to_csv('/kaggle/working/image_size.csv',index=False)
print("Exported!!")

In [ ]:
%%!
zip -r output.zip /kaggle/working/output